# Get data from and save snapshot in S3

In [ ]:
import pandas as pd
import boto3
from tqdm.notebook import trange, tqdm

from utils import MODEL_NAME
from utils.s3 import GLOBAL_DATA_BUCKET, MODELS_BUCKET, read_csv_from_s3

In [ ]:
import os
os.getcwd()

In [ ]:
TABLE_PREFIX = 'Balance_V4'#"v1"

In [ ]:
validated_cases = pd.read_csv("V4_3.csv")#pd.read_csv("Validated_V0_Desriptions.csv")

In [ ]:
validated_cases_old = pd.read_csv("Validated_V0_Desriptions.csv")

In [ ]:
#validated_cases_old

In [ ]:
validated_cases

## Get data from s3

In [ ]:
data_bucket = boto3.resource("s3").Bucket(GLOBAL_DATA_BUCKET)

files = TABLE_PREFIX + "/" + validated_cases["File Names"] + ".csv"

data = []

t = tqdm(files)
for f in t:
    print('f :  ' , f)### <<-- control where crahs happens
    t.set_description(f"loading {f}")
    df = read_csv_from_s3(data_bucket, f, {"parse_dates": ["Block_Time"]})
    df["file_name"] = f
    data.append(df)

data = pd.concat(data, axis=0)

data.shape

## Initial cleanup of column names and datetime data

In [ ]:
data.columns = (
    data.columns.str.replace("[\[\]]", "", regex=True)
    .str.replace(" ", "_", regex=False)
    .str.lower()
)

In [ ]:
data["block_time"] = data["block_time"].dt.tz_localize(None)

In [ ]:
data.columns

In [ ]:
#data.drop(["unnamed:_15", "unnamed:_16"], axis=1, inplace=True) ###<<<-- no more unwanted columns

## Save dataset locally and backup to s3

In [ ]:
data.to_csv("data/raw_data_new_.csv", index=False)

In [ ]:
models_bucket = boto3.resource("s3").Bucket(MODELS_BUCKET)

In [ ]:
models_bucket.upload_file("data/raw_data_new_.csv", f"{MODEL_NAME}/data/raw_data_balance.csv")